# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    my_tokenizer = Tokenizer(num_words=None,
                                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,
                                   split=" ",
                                   char_level=False)
    my_tokenizer.fit_on_texts(x)
    tokenized_data = my_tokenizer.texts_to_sequences(x)
    return tokenized_data, my_tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'the': 1, 'dog': 4, 'prize': 5, 'study': 6, 'this': 7, 'a': 3, 'fox': 8, 'of': 9, 'by': 10, 'jumps': 11, 'lazy': 12, 'quick': 2, 'over': 13, 'won': 14, 'short': 15, 'my': 21, 'brown': 16, 'is': 18, 'lexicography': 19, 'jove': 20, 'sentence': 17}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 16, 8, 11, 13, 1, 12, 4]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 20, 21, 2, 6, 9, 19, 14, 3, 5]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [7, 18, 3, 15, 17]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length == None:
        length = max([len(seq) for seq in x ])
    padded_seq = pad_sequences(x, maxlen=length, padding='post')
    return padded_seq
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 1  2 16  8 11 13  1 12  4]
  Output: [ 1  2 16  8 11 13  1 12  4  0]
Sequence 2 in x
  Input:  [10 20 21  2  6  9 19 14  3  5]
  Output: [10 20 21  2  6  9 19 14  3  5]
Sequence 3 in x
  Input:  [ 7 18  3 15 17]
  Output: [ 7 18  3 15 17  0  0  0  0  0]


C:\Users\Omar\Anaconda3\envs\aind-dl\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((3, 3), 0) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [6]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [7]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [101]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers    
    inputs = Input(input_shape[1:])
    outputs = GRU(1, return_sequences=True) (inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax') )(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    model.summary()

    learning_rate = 0.03
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_65 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_58 (GRU)                 (None, 21, 1)             9         
_________________________________________________________________
time_distributed_34 (TimeDis (None, 21, 344)           688       
Total params: 697.0
Trainable params: 697.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_66 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_59 (GRU)                 (None, 21, 1)             9         
_________________________________________________________________
time_d

In [118]:
logits = simple_rnn_model.predict(tmp_x[:1])[0]
print(logits)
np.argmax(logits)

[[ 0.00270621  0.00271121  0.00264041 ...,  0.00318169  0.00319918
   0.00280717]
 [ 0.00270621  0.00271121  0.00264041 ...,  0.00318169  0.00319918
   0.00280717]
 [ 0.00274082  0.00274493  0.00268651 ...,  0.00312631  0.00314028
   0.00282367]
 ..., 
 [ 0.0028879   0.00288797  0.00288702 ...,  0.00289373  0.00289393
   0.00288922]
 [ 0.00288898  0.00288902  0.00288852 ...,  0.00289204  0.00289215
   0.00288967]
 [ 0.00288954  0.00288956  0.0028893  ...,  0.00289116  0.00289121
   0.00288991]]


68

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [176]:
from keras.layers.embeddings import Embedding
from keras.models import Sequential

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print('input_shape')
    print(input_shape)    
    print('output_sequence_length')
    print(output_sequence_length)
    print('english_vocab_size')
    print(english_vocab_size)
    print('french_vocab_size')
    print(french_vocab_size)
    
    
    # Build the model
    model = Sequential()
    model.add( Embedding(english_vocab_size, 10, input_length=output_sequence_length))#(inputs)
    model.add( GRU(12, return_sequences=True) )
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax') ) )
    model.summary()

    # Compile
    learning_rate = 0.03
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x_reshaped = tmp_x.reshape(tmp_x.shape[0:2])

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x_reshaped.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.fit(tmp_x_reshaped, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x_reshaped[:1])[0], french_tokenizer))

input_shape
(137861, 21)
output_sequence_length
21
english_vocab_size
199
french_vocab_size
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_53 (Embedding)     (None, 21, 10)            1990      
_________________________________________________________________
gru_118 (GRU)                (None, 21, 12)            828       
_________________________________________________________________
time_distributed_86 (TimeDis (None, 21, 344)           4472      
Total params: 7,290.0
Trainable params: 7,290
Non-trainable params: 0.0
_________________________________________________________________
input_shape
(137861, 21)
output_sequence_length
21
english_vocab_size
200
french_vocab_size
346
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, 21, 10)            2000      
__________

In [94]:
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
print((tmp_x[:1])[0])

new jersey est parfois calme en l' automne il il est neigeux en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0  0  0  0  0  0  0]


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [125]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # Build the model
    model = Sequential()
    model.add(Bidirectional(GRU(12, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.summary()

    # Compile
    learning_rate = 0.03
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_23 (Bidirectio (None, 21, 24)            1008      
_________________________________________________________________
time_distributed_51 (TimeDis (None, 21, 344)           8600      
Total params: 9,608.0
Trainable params: 9,608
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_24 (Bidirectio (None, 21, 24)            1008      
_________________________________________________________________
time_distributed_52 (TimeDis (None, 21, 346)           8650      
Total params: 9,658.0
Trainable params: 9,658
Non-trainable params: 0.0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [140]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    # Build the model
    model = Sequential()
    model.add(Bidirectional(GRU(12), input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    #model.add(Activation('softmax'))
    model.summary()

    # Compile
    learning_rate = 0.03
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_37 (Bidirectio (None, 24)                1008      
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 24)            0         
_________________________________________________________________
time_distributed_59 (TimeDis (None, 21, 344)           8600      
Total params: 9,608.0
Trainable params: 9,608.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_38 (Bidirectio (None, 24)                1008      
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 24)            0         
_________________________________________________________________
ti

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
    #adding an embedded layer
    #the size of the input shape is the second value of the tuple input_shape
    input_sequence_length=input_shape[1]    
    newInputs = Embedding(english_vocab_size,input_sequence_length)(inputs)
    #adding a bidirectional layer
    modelFinal=Bidirectional(GRU(output_sequence_length))(newInputs)
    #adding a repeat vector layer to match the output lenght
    modelFinal=RepeatVector(output_sequence_length)(inputs)
    #GRU layer with return sequence=true
    modelFinal = Bidirectional(GRU(output_sequence_length, return_sequences=True))(modelFinal)
    #adding a time distributed layer of french_vocab_size
    modelFinal = TimeDistributed(Dense(french_vocab_size, activation="relu"))(modelFinal)
    #dropout rate to optimize learning
    modelFinal=Dropout(0.25)(modelFinal)
    #adding a time distributed layer of french_vocab_size
    modelFinal = TimeDistributed(Dense(french_vocab_size, activation="relu"))(modelFinal)
    #Activation layer with softmax
    outputs = Activation('softmax')(modelFinal)
    #final layer
    model = Model(inputs=inputs, outputs=outputs)   

In [174]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # Build the model
    model = Sequential()
    # Embedding layer 
    model.add( Embedding(english_vocab_size, 11, input_length=input_shape[1]))
    
    # First recurrent layer
    model.add(Bidirectional(GRU(20)))
    
    # Repeat
    model.add(RepeatVector(output_sequence_length))
    
    # Second recurrent layer
    model.add(Bidirectional(GRU(output_sequence_length, return_sequences=True)))
    
    # Dense layers
    model.add(TimeDistributed(Dense(french_vocab_size*2, activation='relu')))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='relu')))
    
    # Output probabilities
    model.add(Activation('softmax'))
    
    # Show the model
    model.summary()
    
    # Compile
    learning_rate = 0.03
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 15, 11)            2189      
_________________________________________________________________
bidirectional_60 (Bidirectio (None, 40)                3840      
_________________________________________________________________
repeat_vector_26 (RepeatVect (None, 21, 40)            0         
_________________________________________________________________
bidirectional_61 (Bidirectio (None, 21, 42)            7812      
_________________________________________________________________
time_distributed_82 (TimeDis (None, 21, 688)           29584     
_________________________________________________________________
time_distributed_83 (TimeDis (None, 21, 344)           237016    
_________________________________________________________________
activation_3 (Activation)    (None, 21, 344)           0         
Total para

## Prediction (IMPLEMENTATION)

In [181]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import os


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(
                    tmp_x_reshaped.shape,
                    preproc_french_sentences.shape[1],
                    len(english_tokenizer.word_index)+1,
                    len(french_tokenizer.word_index)+1)
    
    # Train the model
    model.fit(tmp_x_reshaped, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)
    
    #Save the model
    model.save(os.path.join('data','model_final.h5'))

    # Print prediction(s)
    #print(logits_to_text(model.predict(tmp_x_reshaped[0])[0], french_tokenizer))
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_56 (Embedding)     (None, 21, 11)            2200      
_________________________________________________________________
bidirectional_66 (Bidirectio (None, 40)                3840      
_________________________________________________________________
repeat_vector_29 (RepeatVect (None, 21, 40)            0         
_________________________________________________________________
bidirectional_67 (Bidirectio (None, 21, 42)            7812      
_________________________________________________________________
time_distributed_90 (TimeDis (None, 21, 692)           29756     
_________________________________________________________________
time_distributed_91 (TimeDis (None, 21, 346)           239778    
_________________________________________________________________
activation_6 (Activation)    (None, 21, 346)           0         
Total para

ValueError: Error when checking : expected embedding_56_input to have shape (None, 21) but got array with shape (2, 15)

In [182]:
model = load_model(os.path.join('data','model_final.h5'))

# Print prediction(s)
print(logits_to_text(model.predict(tmp_x_reshaped[0])[0], french_tokenizer))

ValueError: Error when checking : expected embedding_56_input to have shape (None, 21) but got array with shape (21, 1)

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.